# Generate QAs from Unstructured Data Using AWS Inferentia and Neuron

Are you a machine learning (ML) professional grappling with the challenge of extracting high-quality data from confidential and unstructured formats such as PDFs and HTML? The task of fine-tuning large language models (LLMs) often requires dealing with such complex data, which poses significant challenges in terms of data privacy, processing efficiency, and model accuracy.
In this example, we will show you how to generate question-answers (QAs) from give text strings using open-source models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/transform/transform_huggingface_flow.py).

This article provides a comprehensive guide on leveraging the power of Uniflow on [Amazon Web Services (AWS) Inferentia2 (Inf2) instances](https://aws.amazon.com/machine-learning/inferentia/) for effectively converting unstructured data into a structured format, ideal for private LLM fine-tuning. We delve into the intricacies of handling sensitive information while maintaining confidentiality and data integrity.  Furthermore, we explore the robust capabilities of the AWS Neuron SDK, which plays a pivotal role in enhancing the performance of your ML models. The Neuron SDK is specifically designed to optimize ML workloads on AWS hardware, ensuring you achieve faster processing speeds and reduced operational costs.

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [7]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install transformers accelerate bitsandbytes scipy

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.8 MB/s eta 0:00:000:00:0100:01


### Import dependency

In [8]:
from dotenv import load_dotenv
from IPython.display import display

from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformHuggingFaceConfig, HuggingfaceModelConfig
from uniflow.op.prompt import PromptTemplate, Context

load_dotenv()

False

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class.

In [9]:
sample_instruction = """Generate one question and its corresponding answer based on the context. Following \
the format of the examples below to include context, question, and answer in the response."""

sample_examples = [
        Context(
            context="The quick brown fox jumps over the lazy dog.",
            question="What is the color of the fox?",
            answer="brown."
        ),
        Context(
            context="The quick brown fox jumps over the lazy black dog.",
            question="What is the color of the dog?",
            answer="black."
        )]

guided_prompt = PromptTemplate(
    instruction=sample_instruction,
    few_shot_prompt=sample_examples
)

Second, we craft some dummy sample raw text strings. Below, we build a dataset with 400 text strings.

In [10]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

raw_context_input_400 = raw_context_input * 100

print("sample size of raw context: ", len(raw_context_input_400))

sample size of raw context:  400


Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [11]:

input_data = [
    Context(context=data)
    for data in raw_context_input_400
]

print("sample size of processed input data: ", len(input_data))

input_data[:2]


sample size of processed input data:  400


[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our journey here.')]

### Use LLM to generate data

In this example, we will use the [HuggingfaceModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L39)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here, we pass in our `guided_prompt` to the `HuggingfaceConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [12]:
config = TransformHuggingFaceConfig(
    prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=4, model_name='mistralai/Mistral-7B-Instruct-v0.2', neuron=True))
client = TransformClient(config)

Model chunks found. If you encounter errors, please delete the cache folder `/home/ubuntu/.cache/uniflow/Mistral-7B-Instruct-v0.2-split`
loading neuron model from cache, need to wait for a while...


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [ ]:
output = client.run(input_data)

### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [9]:
import re
from pprint import pprint

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

o = output[0]['output'][0]['response'][0] ## we only postprocess the first output
segments = [segment for segment in re.split(pattern, o) if segment.strip()]
result = {
    "context": segments[-3],
    "question": segments[-2],
    "answer": segments[-1]
}

pprint(result, sort_dicts=False)

{'context': ' We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party infringement and misappropriation.',
 'question': ' Which intellectual property rights does the company actively '
             'seek and protect?\n',
 'answer': ' The company actively seeks and prote

Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `HuggingfaceModelConfig` flow. The answer is "It depends on your data token length, your instance type, your LLM size, etc." In the following experiment, we use an AWS inf2.8xlarge. We still use the above 400 raw data strings `raw_context_input_400`.


Here are the results:

- batch_size = 4
    100%|██████████| 100/100 [07:10<00:00, 4.66s/it]


## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>